## Pointers to send Tensors

The purpose of this is that we want to average gradients before we call .get() therefore we never see anyone's exact gradient; hence further security

In [36]:
import syft as sy
import torch
hook = sy.TorchHook(torch)

### Pointers referencing other pointers

In [37]:
julie = sy.VirtualWorker(hook, id='julie')
toby = sy.VirtualWorker(hook, id='toby')

# create local tensor

tensor1 = torch.tensor([2, 3, 4, 5])

In [38]:
# let's look at what these look like
tensor1

tensor([2, 3, 4, 5])

In [39]:
# send tensor to julie
tensor1_ptr = tensor1.send(julie)
tensor1_ptr

(Wrapper)>[PointerTensor | me:60734232732 -> julie:74310326719]

In [40]:
# send the pointer of the pointer to toby
ptr_to_tensor1_ptr = tensor1_ptr.send(toby)
ptr_to_tensor1_ptr

(Wrapper)>[PointerTensor | me:16472214722 -> toby:60734232732]

In [41]:
# julie still has the tensor itself
julie._objects

{74310326719: tensor([2, 3, 4, 5])}

In [42]:
# toby now has the pointer to julies data
toby._objects

{60734232732: (Wrapper)>[PointerTensor | toby:60734232732 -> julie:74310326719]}

In [43]:
# set it to get the pointer itself
tensor_ptr_backwards = ptr_to_tensor1_ptr.get()
tensor_ptr_backwards

(Wrapper)>[PointerTensor | me:60734232732 -> julie:74310326719]

In [44]:
#
tensor1 = tensor_ptr_backwards.get()
tensor1

tensor([2, 3, 4, 5])

### Arithmetic on Pointer--> Pointer --> Data Object

In [45]:
# ensure toby/julie are empty now, can reset
julie._objects

{}

In [46]:
toby._objects

{}

In [47]:
pointer_to_pointer = torch.tensor([123, 13]).send(julie).send(toby)
y = pointer_to_pointer + pointer_to_pointer

In [48]:
# two tensors
julie._objects

{22008737954: tensor([123,  13]), 16089125378: tensor([246,  26])}

In [49]:
# two pointers to julie's tensors
toby._objects

{41315920398: (Wrapper)>[PointerTensor | toby:41315920398 -> julie:22008737954],
 93978989657: (Wrapper)>[PointerTensor | toby:93978989657 -> julie:16089125378]}

In [50]:
y.get().get()

tensor([246,  26])

In [51]:
# only getting original tensor!
julie._objects

{22008737954: tensor([123,  13])}

In [52]:
toby._objects

{41315920398: (Wrapper)>[PointerTensor | toby:41315920398 -> julie:22008737954]}

In [53]:
pointer_to_pointer.get().get()

tensor([123,  13])

In [54]:
julie._objects

{}

In [55]:
toby._objects

{}

# Section 3.2 - Pointer Chain Ops; call ops on last pointer in chain!

In [56]:
# create new tensor which is a pointer to data on Toby's machine
tensor1 = torch.tensor([1, 2]).send(toby)

In [57]:
toby._objects

{89041931181: tensor([1, 2])}

In [58]:
julie._objects

{}

In [59]:
tensor1 = tensor1.move(julie)

In [60]:
tensor1

(Wrapper)>[PointerTensor | me:61413402627 -> julie:89041931181]

In [61]:
toby._objects

{}

In [62]:
julie._objects

{89041931181: tensor([1, 2])}